In [7]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import env
import acquire as a
from os.path import isfile
from importlib import reload

### Questions:

* 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
* 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
* 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
* 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
* 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
* 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
* 7. Which lessons are least accessed?

In [8]:
#Get the data
reload(a)
curriculum_df = a.wrangle_curriculum_access()
curriculum_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java


### 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [9]:
#Taking all activity for students, not staff
student_df = curriculum_df[curriculum_df.cohort_name != "Staff"]
student_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,PHP
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java


In [10]:
#Taking all activity from students active in course(from start date to end date)
active_df = student_df[(student_df.end_date >= student_df.index) & (student_df.start_date <= student_df.index)]
active_df = active_df[active_df['page'] != '/']
active_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java
2018-01-26 09:56:41,javascript-i/loops,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java
2018-01-26 09:56:46,javascript-i/conditionals,3,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java
2018-01-26 09:56:48,javascript-i/functions,3,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java
2018-01-26 09:56:59,javascript-i/loops,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,Java


In [11]:
#Aggregating pages by student
pages_by_student = active_df.groupby(['id'])['page'].value_counts()
pages_by_student

id   page                                            
2    java-iii                                            50
     mysql                                               32
     java-ii                                             30
     spring                                              29
     jquery                                              28
                                                         ..
981  examples/css/bootstrap-grids.html                    3
     appendix/professional-development/t-block-resume     1
     examples/bootstrap-grid/assets.zip                   1
     html-css/css-ii                                      1
     javascript-ii                                        1
Name: page, Length: 88169, dtype: int64

In [12]:
pages_by_student =pd.DataFrame(pages_by_student)
pages_by_student.head()

page
id page          
2  java-iii    50
   mysql       32
   java-ii     30
   spring      29
   jquery      28

In [13]:
#cleaning it up
pages_by_student.columns = ['page_count']
pages_by_student = pages_by_student.reset_index()
pages_by_student = pages_by_student.groupby('id').sum('page_count')
pages_by_student.nsmallest(n=30, columns='page_count')

,page_count
id,
832,2
278,3
539,4
956,4
388,7
812,7
679,10
941,10
572,11


In [16]:
#Looking at 832, Very little activity, in the middle of the course too, full stack Java Jupiter
user_832 = active_df[active_df.id == 832]
user_832

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2020-12-07 14:22:38,javascript-i,832,69.154.52.98,Jupiter,2020-09-21,2021-03-30,190 days,Java
2020-12-07 14:22:39,html-css,832,69.154.52.98,Jupiter,2020-09-21,2021-03-30,190 days,Java


In [17]:
#Looking at 278, activity towards end of program (very little)
user_278 = active_df[active_df.id == 278]
user_278

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-09-27 14:47:37,java-ii/arrays,278,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,135 days,Java
2018-09-27 14:58:48,java-ii/arrays,278,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,135 days,Java
2018-09-27 14:59:07,java-ii/collections,278,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,135 days,Java


In [18]:
#Looking at 539, may have left the program early on
user_539 = active_df[active_df.id == 539]
user_539

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2019-11-04 16:19:34,toc,539,97.105.19.58,Europa,2019-11-04,2020-04-17,165 days,Java
2019-11-04 16:54:02,html-css,539,97.105.19.58,Europa,2019-11-04,2020-04-17,165 days,Java
2019-11-04 16:54:38,html-css/introduction,539,97.105.19.58,Europa,2019-11-04,2020-04-17,165 days,Java
2019-11-04 16:54:45,html-css/elements,539,97.105.19.58,Europa,2019-11-04,2020-04-17,165 days,Java


In [19]:
#Looking at 812, looks at curriculum towards end of program
user_812 = active_df[active_df.id == 812]
user_812

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2020-11-08 01:45:34,html-css/css-i/selectors-and-properties,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,Java
2020-11-08 01:45:41,html-css/elements,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,Java
2020-11-08 01:45:56,html-css/css-i/introduction,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,Java
2020-11-08 01:46:01,html-css/css-i,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,Java
2020-11-08 01:46:04,html-css/forms,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,Java
2020-11-08 01:46:06,html-css/elements,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,Java
2020-11-08 01:46:07,html-css,812,24.243.49.105,Hyperion,2020-05-26,2020-11-10,168 days,Java


### Many of the users with small page views may have left the program (dates of page views correspond to the begnning of the course. I will reshift those to students who had at least 50 page views

In [20]:
#Students with at least 50 page views
pages_by_student[pages_by_student >= 50].nsmallest(n=10, columns='page_count')

,page_count
id,
56,50.0
46,51.0
543,51.0
962,55.0
36,56.0
858,59.0
57,61.0
45,62.0
975,64.0


In [21]:
#Looking at 56, majority of activity towards end of course
user_56 = active_df[active_df.id == 56]
user_56.head(10)

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-29 09:40:38,spring/fundamentals/relationships,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,Java
2018-01-29 09:40:41,spring/fundamentals/form-model-binding,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,Java
2018-01-29 09:40:44,spring/fundamentals/controllers,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,Java
2018-01-29 09:40:48,spring,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,Java
2018-01-29 12:31:31,java-iii,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,Java
2018-01-29 12:31:35,mysql,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,Java
2018-01-29 12:31:40,mysql/databases,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,Java
2018-01-29 12:32:25,mysql/users,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,Java
2018-01-29 14:57:37,spring,56,97.105.19.61,Sequoia,2017-09-27,2018-02-15,141 days,Java


### Summary Question 3: All of the users identified with very little activity are Java (Web Development) students. Most with very few page views (Less than 10) could be students who did not finish the program. One user (832) only accesses the curriculum twice during the middle of the course. No Data Science students were located in this query